<a href="https://colab.research.google.com/github/GokayToga/VRCM.Net/blob/main/VRCMNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **VRCM.Net**

Virtual Reality Cybersickness Mitigation&Prediction Net, this project is my research project on Exploration of Prediction and Reducement Techniques on Cybersickness in VR Technologies using Machine Learning and Deep Learning

## Data

Gettint the 57 different files of cvs and 5 different volunteers with different parameters of data into one single complete cvs file


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import glob
import os

# Define the path to your CSV files
path = '/content/drive/MyDrive/490_Data'  # Replace with your actual path
all_files = glob.glob(os.path.join(path, "*.csv"))

# Initialize an empty list to hold individual DataFrames
df_list = []

# Iterate over each file
for file in all_files:
    # Extract the base filename without extension
    base_name = os.path.basename(file).replace('.csv', '')

    # Split the filename to extract metric type, volunteer ID, and movement type
    parts = base_name.split('_')
    metric_type = parts[0]
    volunteer_id = parts[1]
    movement_type = parts[2] if len(parts) > 2 else 'normal'  # Default to 'normal' if not specified

    # Read the CSV file into a DataFrame
    df = pd.read_csv(file)

    # Add new columns for volunteer ID, movement type, and metric type
    df['VolunteerID'] = volunteer_id
    df['MovementType'] = movement_type
    df['MetricType'] = metric_type

    # Append the DataFrame to the list
    df_list.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(df_list, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('/content/drive/MyDrive/combined_data.csv', index=False)


## Reinforcement Learning (Random Forests?)

Prediction of Cybersickness level

In [ ]:
# train_and_export_model.py

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Load your dataset
# Assume your CSV has columns: 'fixation_count', 'avg_fixation_duration', 'gaze_variance', and 'cybersickness_score'
data = pd.read_csv("eye_tracking_data.csv")
X = data[['fixation_count', 'avg_fixation_duration', 'gaze_variance']]
y = data['cybersickness_score']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10]
}

rf = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

# Evaluate the model
y_pred = best_model.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))

# Export the trained model to ONNX format for Unity
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]
onnx_model = convert_sklearn(best_model, initial_types=initial_type)
with open("cybersickness_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())


Explanation:

• This script loads the eye tracking dataset, trains a RandomForestRegressor (with hyperparameter tuning), evaluates its performance, and then converts the trained model into an ONNX model for Unity inference.

New training code for the complete cvs file with Cybersickness score

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import joblib

# Load the consolidated CSV file
data_path = "combined_data.csv"  # Update this path as necessary
df = pd.read_csv(data_path)

# Check if the target column is present
if 'CybersicknessScore' not in df.columns:
    raise ValueError("Target column 'CybersicknessScore' not found. Please include it in your CSV file.")

# If necessary, encode any categorical columns (for example, VolunteerID, MovementType, MetricType)
categorical_cols = ['VolunteerID', 'MovementType', 'MetricType']
for col in categorical_cols:
    if col in df.columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))

# Define the features (X) and target (y)
X = df.drop(columns=["CybersicknessScore"])
y = df["CybersicknessScore"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R² Score:", r2)

# Save the trained model for future use
joblib.dump(rf, "rf_cybersickness_model.pkl")

# Function to predict cybersickness score from new data
def predict_cybersickness(new_data):
    """
    new_data should be a DataFrame with the same feature columns as used during training.
    """
    prediction = rf.predict(new_data)
    return prediction

# Example usage:
# Suppose you have new data in the same structure (without the target column)
# new_obs = pd.DataFrame({
#     'FixationCount': [5],
#     'SomeOtherMetric': [0.3],
#     'VolunteerID': [1],
#     'MovementType': [0],
#     'MetricType': [2],
#     ... # include all feature columns used in training
# })
# print("Predicted Cybersickness Score:", predict_cybersickness(new_obs))


## Closed-Loop Feedback System wiht Unity Integration (C#)

Closed-feedback Loop system for manipulating the Fovated Rendering and Depth of Field bluring based on the RL

### Unity integration

In [ ]:
// CybersicknessPredictor.cs
using UnityEngine;
using Unity.Barracuda;

public class CybersicknessPredictor : MonoBehaviour
{
    public NNModel onnxModelAsset; // Assign your ONNX model in the Inspector.
    private Model runtimeModel;
    private IWorker worker;

    void Start()
    {
        // Load the ONNX model and create a worker for inference
        runtimeModel = ModelLoader.Load(onnxModelAsset);
        worker = WorkerFactory.CreateWorker(WorkerFactory.Type.ComputePrecompiled, runtimeModel);
    }

    /// <summary>
    /// Predicts cybersickness based on input features.
    /// </summary>
    /// <param name="fixationCount">Number of fixations</param>
    /// <param name="avgFixationDuration">Average duration of fixations</param>
    /// <param name="gazeVariance">Variance in gaze positions</param>
    /// <returns>Predicted cybersickness score (0-10 scale)</returns>
    public float PredictCybersickness(float fixationCount, float avgFixationDuration, float gazeVariance)
    {
        // Create a tensor with the input features.
        Tensor inputTensor = new Tensor(1, 3);
        inputTensor[0] = fixationCount;
        inputTensor[1] = avgFixationDuration;
        inputTensor[2] = gazeVariance;

        // Execute the model and get the output
        worker.Execute(inputTensor);
        Tensor output = worker.PeekOutput();
        float cybersicknessScore = output[0];

        // Dispose tensors to free up resources
        inputTensor.Dispose();
        output.Dispose();

        return cybersicknessScore;
    }

    void OnDestroy()
    {
        worker.Dispose();
    }
}


Explanation:

• This script loads the ONNX model using Barracuda, sets up an inference worker, and defines a method to take three input features and output a cybersickness prediction.

• You’ll need to ensure the feature order and dimensions match what you defined during training.

### Closed-loop controler

In [ ]:
// VRAdaptiveController.cs
using UnityEngine;

public class VRAdaptiveController : MonoBehaviour
{
    public CybersicknessPredictor predictor; // Reference to the predictor script.
    public float thresholdHigh = 7.0f;
    public float thresholdLow = 3.0f;

    // References to modules that control VR rendering parameters.
    public FoveatedRenderingController foveatedController;
    public DepthOfFieldController depthController;
    // (Optional) Additional module for Field of View adjustments

    void Update()
    {
        // Retrieve eye tracking data from your VR SDK.
        // Replace these dummy methods with actual API calls.
        float fixationCount = GetFixationCount();
        float avgFixationDuration = GetAverageFixationDuration();
        float gazeVariance = GetGazeVariance();

        // Get the cybersickness score using the ML model.
        float cybersicknessScore = predictor.PredictCybersickness(fixationCount, avgFixationDuration, gazeVariance);
        Debug.Log("Cybersickness Score: " + cybersicknessScore);

        // Adaptive control logic:
        if (cybersicknessScore > thresholdHigh)
        {
            // If the score is high, increase mitigation efforts.
            foveatedController.IncreaseFFR();
            depthController.AdjustDOFForComfort();
            // Optionally: adjust FOV if implemented.
        }
        else if (cybersicknessScore < thresholdLow)
        {
            // If the score is low, relax the adjustments.
            foveatedController.DecreaseFFR();
            depthController.ResetDOF();
        }
        // Otherwise, maintain current settings.
    }

    // Dummy methods – replace with your VR API methods to extract eye tracking data.
    float GetFixationCount()
    {
        // TODO: Replace with actual data from your VR headset's eye tracking.
        return 5.0f;
    }

    float GetAverageFixationDuration()
    {
        return 0.2f;
    }

    float GetGazeVariance()
    {
        return 0.05f;
    }
}


Explanation:

• This controller script gathers eye tracking data (you will replace the dummy methods with actual API calls from your VR SDK), calls the prediction model, and then uses simple threshold-based logic to decide if adjustments should be made.

### Foveated Rendering Controller Script

In [ ]:
// FoveatedRenderingController.cs
using UnityEngine;

public class FoveatedRenderingController : MonoBehaviour
{
    public float ffrStrength = 0.5f; // Base FFR strength.

    public void IncreaseFFR()
    {
        ffrStrength = Mathf.Min(ffrStrength + 0.1f, 1.0f);
        ApplyFFRSettings();
    }

    public void DecreaseFFR()
    {
        ffrStrength = Mathf.Max(ffrStrength - 0.1f, 0.0f);
        ApplyFFRSettings();
    }

    private void ApplyFFRSettings()
    {
        // Implement the call to your VR system’s FFR settings.
        // For example, if your VR SDK allows adjusting resolution in peripheral areas:
        Debug.Log("Applying FFR Strength: " + ffrStrength);
        // TODO: Integrate with actual rendering settings.
    }
}


Explanation:

• This script defines methods to increase or decrease the strength of the foveated rendering effect. Adjust the internal logic to suit your VR system’s API.

### Depth of Field Controller Script

In [ ]:
// DepthOfFieldController.cs
using UnityEngine;

public class DepthOfFieldController : MonoBehaviour
{
    public float currentDOFIntensity = 0.5f;

    public void AdjustDOFForComfort()
    {
        // Increase DOF intensity (or adjust parameters) to reduce peripheral distractions.
        currentDOFIntensity = Mathf.Min(currentDOFIntensity + 0.1f, 1.0f);
        ApplyDOFSettings();
    }

    public void ResetDOF()
    {
        // Reset DOF settings to a default comfortable value.
        currentDOFIntensity = 0.5f;
        ApplyDOFSettings();
    }

    private void ApplyDOFSettings()
    {
        // Apply the DOF adjustments to your camera's post-processing profile.
        Debug.Log("DOF intensity set to " + currentDOFIntensity);
        // TODO: Link with your post-processing stack or DOF effect settings.
    }
}


Explanation:

• This script similarly adjusts the intensity of a Depth of Field effect based on the cybersickness prediction. Customize the effect parameters based on your VR scene’s needs.

## Finalization

In Unity:

* Import your ONNX model asset and add it to a GameObject with the CybersicknessPredictor component.

* Create separate GameObjects for the FoveatedRenderingController and DepthOfFieldController and attach their scripts.

* Attach the VRAdaptiveController script to a central controller GameObject and assign the above components in the Inspector.

Real-Time Operation:

* During runtime, the VRAdaptiveController will call your dummy eye tracking data methods (replace these with actual sensor calls).

* The predictor makes a continuous prediction which the controller uses to adjust rendering parameters via your adaptive controllers.

Testing & Iteration:

* Validate the prediction output and observe the dynamic adjustments.

* Use debugging logs to fine-tune threshold values, increment steps, and overall system responsiveness.
